In [1]:
from sympy import *

In [28]:
x = symbols('x')
y = Function('y')(x)
xi = Function('xi')(x,y)
eta = Function('eta')(x,y)

In [21]:
Derivative(xi,x,2) ## partial derivative

Derivative(xi(x, y(x)), (x, 2))

In [26]:
diff(xi,x,2) ## total partial derivative

(Derivative(xi(x, y(x)), (y(x), 2))*Derivative(y(x), x) + Subs(Derivative(xi(x, _xi), _xi, x), _xi, y(x)))*Derivative(y(x), x) + Derivative(xi(x, y(x)), y(x))*Derivative(y(x), (x, 2)) + Derivative(y(x), x)*Subs(Derivative(xi(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x) + Subs(Derivative(xi(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x)

In [ ]:
f = 

In [ ]:
class Var:
    def __init__(self, var, derivatives):
    self.v = var
    self.d = derivatives

# Fix Torres' issues so he is not mad anymore :c

In [1]:
import sympy as sp
import numpy as np
import re
import ipdb
from utils.algebra import str_eqn_to_dict_eqn

In [2]:
x = sp.symbols('x')
y = sp.Function('y')(x)
list_indep = [x]
list_dep = [y]
list_cte = []

list_var = list_indep + list_dep
list_all = list_cte + list_indep + list_dep

In [3]:
def parens(s):
    i = s.count(')') - 1
    groups = s[s.find('('):].split(')')
    return (')'.join(groups[:i]) + ')')

In [5]:
def str_to_dict(f, term, arr_pow, arr_deriv, list_all, list_var):
    if f.is_Mul:
        for i in f.args:
            str_to_dict(i, term, arr_pow, arr_deriv, list_all, list_var)
    else:
        if f.args == ():
            if f.is_Integer:
                term['coefficient'] = f
            else:
                idx = np.where(list_all == f)
                p = 1
                arr_pow[idx] = p

        if f.is_Function:
            idx = np.where(list_all == f)
            p = 1
            arr_pow[idx] = p
            if idx[0].size == 0:
                term['variable'] = str(f).split('(')[0]

        if f.is_Pow:
            var = f.args[0]
            if var.is_Derivative:
                term['variable'] = var.args[0]
                for v in var.args[1:]:
                    idx = np.where(list_var == v[0])
                    arr_deriv[idx] = v[1]
            else:
                idx = np.where(list_all == f.args[0])
                arr_pow[idx] = f.args[1]

        if type(f) == type(sp.Subs(list_var[0],
                                   list_var[0], list_var[0])):
            s = str(f)
            if s.endswith('))'):
                if '(' in re.split(',', s)[-1]:
                    subs = re.split(',',s)[-2].strip(' ')
                    subs_t = re.split(',',s)[-1].strip(')').strip(' ')    # Changed
                    subs_t = subs_t + ')'
                else:
                    subs = re.split(',', s)[-3].strip(' ')
                    subs_t = re.split(',', s)[-2] + ',' + re.split(',', s)[-1]
                    subs_t = subs_t[:-1].strip(' ')
            else:
                subs = re.split(',',s)[-2].strip(' ')
                subs_t = re.split(',',s)[-1].strip(')').strip(' ')
            if ')' in subs:
                subs = subs.strip(')')
            s = s.replace(subs, subs_t)
            f = sp.sympify(parens(s).strip('('))
            if isinstance(f, tuple):
                f = f[0]

        if f.is_Derivative:
            term['variable'] = str(f.args[0]).split("(")[0]
            for v in f.args[1:]:
                idx = np.where(list_var == v[0])
                arr_deriv[idx] = v[1]

    term['constants'] = list(arr_pow.astype(int))
    term['derivatives'] = list(arr_deriv.astype(int))
    return term


In [6]:
dict_det_eqn = {'Derivative(y(x),x)*Derivative(y(x),(x,2))': ['-xix(x,y(x))*y(x)',
  '+3*y(x)^2*Derivative(xix(x,y(x)),y(x))'],
 'Derivative(y(x),(x,4))': ['-xix(x,y(x))*y(x)'],
 'Derivative(y(x),x)*Derivative(y(x),(x,3))': ['-2*xix(x,y(x))'],
 'Derivative(y(x),(x,2))^2': ['-xix(x,y(x))'],
 'Derivative(y(x),x)^2': ['+y(x)*Derivative(etay(x,y(x)),(y(x),2))',
  '-y(x)*Subs(Derivative(xix(_xi_1,y(x)),_xi_1,y(x)),_xi_1,x)',
  '-y(x)*Subs(Derivative(xix(x,_xi),_xi,x),_xi,y(x))'],
 'Derivative(y(x),x)^3': ['-y(x)*Derivative(xix(x,y(x)),(y(x),2))'],
 'Derivative(y(x),x)': ['-y(x)*Subs(Derivative(xix(_xi_1,y(x)),(_xi_1,2)),_xi_1,x)',
  '+y(x)*Subs(Derivative(etay(_xi_1,y(x)),_xi_1,y(x)),_xi_1,x)',
  '+y(x)*Subs(Derivative(etay(x,_xi),_xi,x),_xi,y(x))'],
 'Derivative(y(x),(x,2))': ['-etay(x,y(x))*y(x)',
  '-y(x)^2*Derivative(etay(x,y(x)),y(x))',
  '+2*y(x)^2*Subs(Derivative(xix(_xi_1,y(x)),_xi_1),_xi_1,x)'],
 'lonely_terms': ['+y(x)*Subs(Derivative(etay(_xi_1,y(x)),(_xi_1,2)),_xi_1,x)']}

In [7]:
det_eqn = []
for eqn in dict_det_eqn.values():
    aux_list = []
    for str_term in eqn:
        arr_pow = np.zeros(len(list_all))
        arr_deriv = np.zeros(len(list_var))
        term = {"coefficient": 1, "constants": None,
                "derivatives": None, "variable": None}
        aux_list.append(str_to_dict(sp.sympify(str_term), term, arr_pow,
              arr_deriv, np.array(list_all), np.array(list_var)))
    det_eqn.append(aux_list)
keys = list(np.arange(len(det_eqn)))
ret = dict(zip(keys, det_eqn))

Subs(Derivative(xix(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x)
Subs(Derivative(xix(x, _xi), _xi, x), _xi, y(x))
Subs(Derivative(xix(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x)
Subs(Derivative(etay(_xi_1, y(x)), _xi_1, y(x)), _xi_1, x)
Subs(Derivative(etay(x, _xi), _xi, x), _xi, y(x))
Subs(Derivative(xix(_xi_1, y(x)), _xi_1), _xi_1, x)
Subs(Derivative(etay(_xi_1, y(x)), (_xi_1, 2)), _xi_1, x)


In [8]:
ret

{0: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [0, 0],
   'variable': 'xix'},
  {'coefficient': 3,
   'constants': [0, 2],
   'derivatives': [0, 1],
   'variable': 'xix'}],
 1: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [0, 0],
   'variable': 'xix'}],
 2: [{'coefficient': -2,
   'constants': [0, 0],
   'derivatives': [0, 0],
   'variable': 'xix'}],
 3: [{'coefficient': -1,
   'constants': [0, 0],
   'derivatives': [0, 0],
   'variable': 'xix'}],
 4: [{'coefficient': 1,
   'constants': [0, 1],
   'derivatives': [0, 2],
   'variable': 'etay'},
  {'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [1, 1],
   'variable': 'xix'},
  {'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [1, 1],
   'variable': 'xix'}],
 5: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [0, 2],
   'variable': 'xix'}],
 6: [{'coefficient': -1,
   'constants': [0, 1],
   'derivatives': [2, 0],
   'variable': 'xix'},
  {'coefficient': 1,
   